In [2]:
# Run this cell once to setup the session/environment
!rm -rf sample_data
!rm -rf fp-dataset-artifacts
!git clone https://github.com/dtsal3/fp-dataset-artifacts/

Cloning into 'fp-dataset-artifacts'...
remote: Enumerating objects: 79, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 79 (delta 17), reused 51 (delta 13), pack-reused 23
Unpacking objects: 100% (79/79), done.


In [3]:
import os
os.chdir("/content/fp-dataset-artifacts")
!git pull 
os.chdir("/content/")

Already up to date.


In [4]:
!pip install -r fp-dataset-artifacts/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 4.2 MB/s 
     |████████████████████████████████| 441 kB 67.6 MB/s 
     |████████████████████████████████| 7.6 MB 56.2 MB/s 
     |████████████████████████████████| 182 kB 72.1 MB/s 
     |████████████████████████████████| 115 kB 86.4 MB/s 
     |████████████████████████████████| 212 kB 76.0 MB/s 
     |████████████████████████████████| 95 kB 5.5 MB/s 
     |████████████████████████████████| 127 kB 75.4 MB/s 
     |████████████████████████████████| 115 kB 92.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: dill
    Found existing installation: dill 0.3.6
    Uninstalling dill-0.3.6:
      Successfully uninstalled dill-0.3.6


In [5]:
!pip install checklist
!jupyter nbextension install --py --sys-prefix checklist.viewer
!jupyter nbextension enable --py --sys-prefix checklist.viewer
!python -m spacy download en_core_web_sm


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.1 MB 4.0 MB/s 
  Using cached jupyter-1.0.0-py2.py3-none-any.whl (2.7 kB)
     |████████████████████████████████| 22.3 MB 1.4 MB/s 
     |████████████████████████████████| 167 kB 49.8 MB/s 
  Using cached jedi-0.18.2-py2.py3-none-any.whl (1.6 MB)
  Using cached qtconsole-5.4.0-py3-none-any.whl (121 kB)
     |████████████████████████████████| 81 kB 9.8 MB/s 
     |████████████████████████████████| 5.6 MB 15.8 MB/s 
     |████████████████████████████████| 5.6 MB 19.3 MB/s 
     |████████████████████████████████| 348 kB 35.9 MB/s 
     |████████████████████████████████| 100 kB 10.2 MB/s 
     |████████████████████████████████| 4.0 MB 71.0 MB/s 
  Using cached QtPy-2.3.0-py3-none-any.whl (83 kB)
  Created wheel for checklist: filename=checklist-0.0.11-py3-none-any.whl size=12165635 sha256=e7345211d42c68853c0e2099dac28a985af556c062e0d29de0bf5264ca00ab

In [6]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [30]:
import checklist
import spacy
import itertools

import numpy as np
import torch
import transformers
from transformers import AutoModelForQuestionAnswering, AutoTokenizer

from checklist.test_suite import TestSuite
import checklist.editor
import checklist.text_generation
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect
from checklist.test_suite import TestSuite
from checklist.perturb import Perturb
from transformers import pipeline

from datasets import Dataset
import datasets
squad_adv_addSent = datasets.load_dataset('squad_adversarial', 'AddOneSent')
nq_data = datasets.load_dataset("json",data_files="./fp-dataset-artifacts/datasets/nq_train.json", field='data')
squad = datasets.load_dataset('squad')
squad_v2 = datasets.load_dataset('squad_v2')


editor = checklist.editor.Editor()
editor.tg

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating validation split:   0%|          | 0/1787 [00:00<?, ? examples/s]

Dataset squad_adversarial downloaded and prepared to /root/.cache/huggingface/datasets/squad_adversarial/AddOneSent/1.1.0/e9df92c060f50eb529284b303c504bf4359ba37944faebe7a16a91b7d534e946. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

...

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

Dataset squad_v2 downloaded and prepared to /root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [8]:

model_dir = "./fp-dataset-artifacts/trained_model/"

model_nlp = pipeline('question-answering', model=AutoModelForQuestionAnswering.from_pretrained(model_dir), tokenizer=AutoTokenizer.from_pretrained(model_dir))

Just some fun analysis to try to get some intuition in what might be going on:


In [9]:
new_context = """Christian Pulisic assured his United States teammates he'd be ready for Saturday's match against the Netherlands after picking up an abdominal injury in his team's Group B match win over Iran on Tuesday.

                Pulisic was involved in a hard collision with Iran keeper Alireza Beiranvand shortly before half-time as he tapped in a headed cross from Sergino Dest to put the US up 1-0 and help earn the Americans a spot in the knockout rounds.

                The Chelsea winger said in a social media post after the game: "I'll be ready Saturday, don't worry," while teammate Weston McKennie said: "I sent him a text and checked on him, and he said, `Best believe I'll be ready on Saturday.'"

                Pulisic lay on the ground in the goalmouth for a few minutes following the collision before leaving the field and receiving attention from the team's trainers and then coming back on to finish the first half.

                Brenden Aaronson came on for Pulisic after half-time, with US Soccer saying after the match he had been taken to the hospital and diagnosed with a pelvic contusion that was termed "day to day."

                "Obviously we're very thankful that he threw his body there,'' McKennie said after the match. "At the end it was a heart-drop sinking moment, but we got it done and we're excited to still be here.''

                US coach Gregg Berhalter praised Pulisic's hard work and toughness after the match."""

In [10]:
print(model_nlp({"context": new_context,"question":"Who did Christian Pulisic crash into"})) # easy, gets it
print(model_nlp({"context": new_context,"question":"Who did Christian Pulisic beat"})) # "beat" is not in the context, isn't generalizing to new data well
print(model_nlp({"context": new_context,"question":"Who did Christian Pulisic win over"}))  # win over in context, but have to go from pulisic -> his team -> over iran, kind of multi-hop ish
print(model_nlp({"context": new_context,"question":"Who did the United States beat"})) # again, more direct question but "beat" is new to the model given the context
print(model_nlp({"context": new_context,"question":"Who did the United States win over"})) # I think the "Who" is trying to identify a person, tricking the model 
print(model_nlp({"context": new_context,"question":"Which team did the United States win over"})) # Answers the super direct, with words from context question correct
print(model_nlp({"context": new_context,"question":"Which team does the United States face next"})) # I think this might be right just because Netherlands appears before Iran
print(model_nlp({"context": new_context,"question":"What injury does Christian Pulisic have"})) # correct
print(model_nlp({"context": new_context,"question":"Did Weston McKennie get hurt"})) # answers the question but for the wrong subject
print(model_nlp({"context": new_context,"question":"Did Christian Pulisic get hurt"})) # kind of indirect answer, but kind of right which is cool
print(model_nlp({"context": new_context,"question":"What teams are playing in the World Cup"})) # can't give multiple answers in this format
print(model_nlp({"context": new_context,"question":"What emotion did Weston McKennie feel"})) # better answer than I expected, but was hoping for excited or thankful
print(model_nlp({"context": new_context,"question":"Which team played against the United States but did not win"})) # too complicated? is it the negation?


{'score': 0.8838810920715332, 'start': 279, 'end': 297, 'answer': 'Alireza Beiranvand'}
{'score': 0.11321718245744705, 'start': 97, 'end': 112, 'answer': 'the Netherlands'}
{'score': 0.08052878826856613, 'start': 26, 'end': 53, 'answer': 'his United States teammates'}
{'score': 0.22671659290790558, 'start': 0, 'end': 17, 'answer': 'Christian Pulisic'}
{'score': 0.3520931005477905, 'start': 0, 'end': 17, 'answer': 'Christian Pulisic'}
{'score': 0.19984397292137146, 'start': 187, 'end': 191, 'answer': 'Iran'}
{'score': 0.5213747620582581, 'start': 97, 'end': 112, 'answer': 'the Netherlands'}
{'score': 0.5761856436729431, 'start': 133, 'end': 149, 'answer': 'abdominal injury'}
{'score': 0.06418806314468384, 'start': 651, 'end': 701, 'answer': "he said, `Best believe I'll be ready on Saturday.'"}
{'score': 0.014556078240275383, 'start': 0, 'end': 17, 'answer': 'Christian Pulisic'}
{'score': 0.07759753614664078, 'start': 97, 'end': 112, 'answer': 'the Netherlands'}
{'score': 0.0810583382844

In [11]:
print(model_nlp({"context": "Samantha is slower than Jordan.","question":"Who is less slow?"})) # doesn't deal with negation well?
c={"context": "Samantha is slower than Jordan.","question":"Who is less slow?"}
model_nlp(c)['answer']

{'score': 0.9248237013816833, 'start': 0, 'end': 8, 'answer': 'Samantha'}


'Samantha'

Try to do some checklist generation for real:


In [12]:
def format_squad_with_context(x, pred, conf, label=None, *args, **kwargs):
    c, q = x
    ret = 'C: %s\nQ: %s\n' % (c, q)
    if label is not None:
        ret += 'A: %s\n' % label
    ret += 'P: %s\n' % pred
    return ret

def format_squad(x, pred, conf, label=None, *args, **kwargs):
    
    try:
      c, q, s = x
    except:
      c, q = x
    ret = 'Q: %s\n' % (q)
    if label is not None:
        ret += 'A: %s\n' % label
    ret += 'P: %s\n' % pred
    return ret

import json
def load_squad(fold='validation'):
    answers = []
    data = []
    ids = []
    files = {
        'validation': 'fp-dataset-artifacts/datasets/squad-dev-v2.0.json',
        'train': 'fp-dataset-artifacts/datasets/squad-train-v2.0.json',
        }
    f = json.load(open(files[fold]))
    for t in f['data']:
        for p in t['paragraphs']:
            context = p['context']
            for qa in p['qas']:
                data.append({'passage': context, 'question': qa['question'], 'id': qa['id']})
                if qa["is_impossible"]:
                  answers.append(("true",set([(x['text'], x['answer_start']) for x in qa['plausible_answers']])))
                else:
                  answers.append(("false", set([(x['text'], x['answer_start']) for x in qa['answers']])))
    return data, answers

In [13]:
import pickle
data, answers =  load_squad('validation')
pairs = [(x['passage'], x['question'], x['id']) for x in data]

In [14]:
pairs[0]

('The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.',
 'In what country is Normandy located?',
 '56ddde6b9a695914005b9628')

In [15]:
nlp = spacy.load("en_core_web_sm")

processed_pairs = []
for pair in pairs[0:10]:
  passage_doc = nlp(pair[0])
  question_doc = nlp(pair[1])
  id = pair[2]
  processed_pairs.append((passage_doc, question_doc, id))

In [16]:
print(processed_pairs[0])
print(pairs[0])

(The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries., In what country is Normandy located?, '56ddde6b9a695914005b9628')
('The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended 

In [17]:
print(len(pairs))

11873


In [18]:
from spacy import displacy

displacy.render(processed_pairs[0][1], style = "ent",jupyter = True)
entities = []
for pair in processed_pairs:
  entities.append((pair[1], [(i, i.label_) for i in pair[1].ents ]))
entities

[(In what country is Normandy located?, [(Normandy, 'PERSON')]),
 (When were the Normans in Normandy?,
  [(Normans, 'NORP'), (Normandy, 'PERSON')]),
 (From which countries did the Norse originate?, [(Norse, 'FAC')]),
 (Who was the Norse leader?, [(Norse, 'ORG')]),
 (What century did the Normans first gain their separate identity?,
  [(Normans, 'NORP'), (first, 'ORDINAL')]),
 (Who gave their name to Normandy in the 1000's and 1100's,
  [(Normandy, 'PERSON'), (1100, 'DATE')]),
 (What is France a region of?, [(France, 'GPE')]),
 (Who did King Charles III swear fealty to?, [(Charles III, 'PERSON')]),
 (When did the Frankish identity emerge?, []),
 (Who was the duke in the battle of Hastings?, [(Hastings, 'PERSON')])]

In [19]:
[pair[1] for pair in processed_pairs]

[In what country is Normandy located?,
 When were the Normans in Normandy?,
 From which countries did the Norse originate?,
 Who was the Norse leader?,
 What century did the Normans first gain their separate identity?,
 Who gave their name to Normandy in the 1000's and 1100's,
 What is France a region of?,
 Who did King Charles III swear fealty to?,
 When did the Frankish identity emerge?,
 Who was the duke in the battle of Hastings?]

In [20]:
data[9], answers[9]

({'passage': 'The Norman dynasty had a major political, cultural and military impact on medieval Europe and even the Near East. The Normans were famed for their martial spirit and eventually for their Christian piety, becoming exponents of the Catholic orthodoxy into which they assimilated. They adopted the Gallo-Romance language of the Frankish land they settled, their dialect becoming known as Norman, Normaund or Norman French, an important literary language. The Duchy of Normandy, which they formed by treaty with the French crown, was a great fief of medieval France, and under Richard I of Normandy was forged into a cohesive and formidable principality in feudal tenure. The Normans are noted both for their culture, such as their unique Romanesque architecture and musical traditions, and for their significant military accomplishments and innovations. Norman adventurers founded the Kingdom of Sicily under Roger II after conquering southern Italy on the Saracens and Byzantines, and an 

In [21]:
data[6], answers[6]

({'passage': 'The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.',
  'question': 'What is France a region of?',
  'id': '5ad39d53604f3c001a3fe8d2'},
 ('true', {('Normandy', 137)}))

In [22]:
suite = TestSuite()

In [23]:
## Vocabulary 
adj = ['old', 'smart', 'tall', 'young', 'strong', 'short', 'tough', 'cool', 'fast', 'nice', 'small', 'dark', 'wise', 'rich', 'great', 'weak', 'high', 'slow', 'strange', 'clean']
adj = [(x.rstrip('e'), x) for x in adj]


In [24]:
from checklist.pred_wrapper import PredictorWrapper
pred = lambda x: ([model_nlp({"context":x[0], "question":x[1]})['answer']], [model_nlp({"context":x[0], "question":x[1]})['score']])
testing_example = [("Samantha is slower than Jordan.","Who is less slow?")]

def predict_and_score_fun(input):
    
    preds = []
    scores = []
    for i in input:
      context = i[0] 
      question = i[1]
      pred = model_nlp({"context":context,"question":question})
      preds.append(pred['answer'])
      scores.append(pred['score'])
    return (preds, scores)

new_pred = PredictorWrapper.wrap_predict(predict_and_score_fun)

print(predict_and_score_fun(testing_example))

(['Samantha'], [0.9248237013816833])


In [26]:
'''
print("first data point in T")
print(t.labels[0])
print(t.data[0])
print("second data point in T")
print(t.labels[1])
print(t.data[1])
'''
def checklist_to_squad(name, new_data, current_data=None, num_samples=None):

  ids = []
  titles = []
  contexts = []
  questions = []
  answers = []

  for example_group_idx in range(len(new_data.data)):
    for example_idx in range(len(new_data.data[example_group_idx])):
      #print("our indices: ", example_group_idx, example_idx, "in ", len(new_data.data), len(example))
      example_data = new_data.data[example_group_idx][example_idx]
      context, question = example_data
      example_label = new_data.labels[example_group_idx][example_idx]
      ids.append("a"+str(example_group_idx)+str(example_idx))
      titles.append("name")
      contexts.append(context)
      questions.append(question)

      answers.append({'text':[example_label], "answer_start":[context.find(example_label)]})

  my_data = {
      "id": ids,
      "title": titles,
      "context": contexts,
      "question": questions,
      "answers": answers
    
  }
  checklist_data = Dataset.from_dict(my_data, features=squad["train"].features)
  if num_samples != None:
    selection = random.sample(range(checklist_data.num_rows), num_samples)
    checklist_data = checklist_data.select(selection)
  if current_data == None:
    return checklist_data
  else:
    return datasets.concatenate_datasets([current_data, checklist_data])

#checklist_data = checklist_to_squad("Profession_vs_nationality", t)
#print(checklist_data)

In [210]:
t = editor.template(
    [(
    '{first_name} is {adj[0]}er than {first_name1}.',
    'Who is less {adj[1]}?'
    ),(
    '{first_name} is {adj[0]}er than {first_name1}.',
    'Who is {adj[0]}er?'
    )
    ],
    labels = ['{first_name1}','{first_name}'],
    adj=adj,
    remove_duplicates=True,
    nsamples=50,
    save=True
    )
name = 'A is COMP than B. Who is more / less COMP?'
description = ''
test = MFT(**t, name=name, description=description, capability='Vocabulary')
test.run(predict_and_score_fun)
test.summary(n=3, format_example_fn=format_squad_with_context)
suite.add(test, overwrite=True)
checklist_data = checklist_to_squad("more_less_comp", t)
checklist_data_full = checklist_to_squad("more_less_comp", t)


Predicting 100 examples
Test cases:      50
Fails (rate):    50 (100.0%)

Example fails:
C: Betty is stronger than Steven.
Q: Who is less strong?
A: Steven
P: Betty


----
C: Billy is smarter than Karen.
Q: Who is less smart?
A: Karen
P: Billy


----
C: Hugh is higher than Jack.
Q: Who is less high?
A: Jack
P: Hugh


----


In [211]:
checklist_data

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 100
})

In [212]:
def crossproduct(t):
    # takes the output of editor.template and does the cross product of contexts and qas
    ret = []
    ret_labels = []
    for x in t.data:
        cs = x['contexts']
        qas = x['qas']
        d = list(itertools.product(cs, qas))
        ret.append([(x[0], x[1][0]) for x in d])
        ret_labels.append([x[1][1] for x in d])
    t.data = ret
    t.labels = ret_labels
    return t

def clean(string):
    return string.lstrip('[a,the,an,in,at] ').rstrip('.')

def expect_squad(x, pred, conf, label=None, meta=None):
    return clean(pred) == clean(label)
expect_squad = Expect.single(expect_squad)

state = editor.suggest('John is very {mask} about the project.')[:20]
print(', '.join(editor.suggest('John is {mask} {state} about the project.', state=state)[:30]))
very = ['very', 'extremely', 'really', 'quite', 'incredibly', 'particularly', 'highly', 'super']
somewhat = ['a little', 'somewhat', 'slightly', 'mildly']

very, pretty, extremely, also, still, quite, more, really, not, clearly, fairly, incredibly, particularly, now, understandably, rather, cautiously, surprisingly, certainly, feeling, so, especially, definitely, generally, most, highly, super, reportedly, being, obviously


In [213]:
t = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} is {very} {s} about the project. {first_name1} is {s} about the project.',
            '{first_name1} is {s} about the project. {first_name} is {very} {s} about the project.',
            '{first_name} is {s} about the project. {first_name1} is {somewhat} {s} about the project.',
            '{first_name1} is {somewhat} {s} about the project. {first_name} is {s} about the project.',
            '{first_name} is {very} {s} about the project. {first_name1} is {somewhat} {s} about the project.',
            '{first_name1} is {somewhat} {s} about the project. {first_name} is {very} {s} about the project.',
        ],
        'qas': [
            (
                'Who is most {s} about the project?',
                '{first_name}'
            ), 
            (
                'Who is least {s} about the project?',
                '{first_name1}'
            ), 
            
        ]
        
    },
    s = state,
    very=very,
    somewhat=somewhat,
    remove_duplicates=True,
    nsamples=50,
    save=True
    ))
name = 'Intensifiers (very, super, extremely) and reducers (somewhat, kinda, etc)?'
desc = ''
test = MFT(**t, name=name, description=desc, capability='Vocabulary')
test.run(predict_and_score_fun)
test.summary(n=3, format_example_fn=format_squad_with_context)
suite.add(test, overwrite=True)
checklist_data = checklist_to_squad("intensifiers_and_reducers", t, checklist_data, 100)
checklist_data_full = checklist_to_squad("intensifiers_and_reducers", t, checklist_data_full)

Predicting 588 examples
Test cases:      49
Fails (rate):    49 (100.0%)

Example fails:
C: Anne is passionate about the project. Nick is somewhat passionate about the project.
Q: Who is most passionate about the project?
A: Anne
P: Nick

C: Anne is very passionate about the project. Nick is somewhat passionate about the project.
Q: Who is most passionate about the project?
A: Anne
P: Nick

C: Anne is very passionate about the project. Nick is passionate about the project.
Q: Who is most passionate about the project?
A: Anne
P: Nick


----
C: Victoria is excited about the project. Harry is really excited about the project.
Q: Who is least excited about the project?
A: Victoria
P: Harry

C: Harry is really excited about the project. Victoria is excited about the project.
Q: Who is most excited about the project?
A: Harry
P: Victoria

C: Harry is really excited about the project. Victoria is mildly excited about the project.
Q: Who is most excited about the project?
A: Harry
P: Victoria


In [214]:
checklist_data

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 200
})

In [215]:
model_nlp({'context': "Samuel is very particular about the project. Keith is somewhat particular about the project.", "question":"Who is most particular about the project?"})

{'score': 0.5773154497146606, 'start': 45, 'end': 50, 'answer': 'Keith'}

In [216]:
animals = ['dog', 'cat', 'bull', 'cow', 'fish', 'serpent', 'snake', 'lizard', 'hamster', 'rabbit', 'guinea pig', 'iguana', 'duck']
vehicles = ['car', 'truck', 'train', 'motorcycle', 'bike', 'firetruck', 'tractor', 'van', 'SUV', 'minivan']
t = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} has {a:animal} and {a:vehicle}.',
            '{first_name} has {a:vehicle} and {a:animal}.',
        ],
        'qas': [
            (
                'What animal does {first_name} have?',
                '{animal}'
            ), 
            (
                'What vehicle does {first_name} have?',
                '{vehicle}'
            ), 
            
        ]
        
    },
    animal=animals,
    vehicle=vehicles,
    remove_duplicates=True,
    nsamples=50,
    save=True
    ))
name = 'Animal vs Vehicle'
test = MFT(**t, name=name, description='', capability='Taxonomy', expect=expect_squad)
test.run(predict_and_score_fun)
test.summary(n=3, format_example_fn=format_squad_with_context)
suite.add(test, overwrite=True)
checklist_data = checklist_to_squad("animal_vs_vehicle", t, checklist_data, 100)
checklist_data_full = checklist_to_squad("animal_vs_vehicle", t, checklist_data_full)

Predicting 200 examples
Test cases:      50
Fails (rate):    40 (80.0%)

Example fails:
C: William has a rabbit and a van.
Q: What vehicle does William have?
A: van
P: a rabbit and a van


----
C: Bill has a dog and a truck.
Q: What vehicle does Bill have?
A: truck
P: a dog and a truck


----
C: Helen has a snake and a SUV.
Q: What vehicle does Helen have?
A: SUV
P: a snake and a SUV


----


In [217]:
professions = editor.suggest('{first_name} works as {a:mask}.')[:30]
professions += editor.suggest('{first_name} {last_name} works as {a:mask}.')[:30]
professions = list(set(professions))
if 'translator' in professions:
    professions.remove('translator')

t = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} is {a:nat} {prof}.',
            '{first_name} is {a:prof}. {first_name} is {nat}.',
            '{first_name} is {nat}. {first_name} is {a:prof}.',
            '{first_name} is {nat} and {a:prof}.',
            '{first_name} is {a:prof} and {nat}.',
        ],
        'qas': [
            (
                'What is {first_name}\'s job?',
                '{prof}'
            ), 
            (
                'What is {first_name}\'s nationality?',
                '{nat}'
            ), 
            
        ]
        
    },
    nat = editor.lexicons['nationality'][:10],
    prof=professions,
    remove_duplicates=True,
    nsamples=500,
    save=True,
    ))
name = 'Profession vs nationality'
test = MFT(**t, name=name, expect=expect_squad, description='',  capability='Taxonomy')
test.run(predict_and_score_fun)
test.summary(n=3, format_example_fn=format_squad_with_context)
suite.add(test, overwrite=True)
checklist_data = checklist_to_squad("profession_vs_nationality", t, checklist_data, 100)
checklist_data_full = checklist_to_squad("profession_vs_nationality", t, checklist_data_full)

Predicting 5000 examples
Test cases:      500
Fails (rate):    177 (35.4%)

Example fails:
C: Sandra is Nigerian and a DJ.
Q: What is Sandra's job?
A: DJ
P: Nigerian and a DJ


----
C: Colin is an Indian model.
Q: What is Colin's job?
A: model
P: Indian model

C: Colin is Indian and a model.
Q: What is Colin's job?
A: model
P: Indian and a model

C: Colin is a model and Indian.
Q: What is Colin's job?
A: model
P: model and Indian


----
C: Christopher is a Japanese agent.
Q: What is Christopher's job?
A: agent
P: Japanese agent


----


In [218]:
comp_pairs = [('better', 'worse'), ('older', 'younger'), ('smarter', 'dumber'), ('taller', 'shorter'), ('bigger', 'smaller'), ('stronger', 'weaker'), ('faster', 'slower'), ('darker', 'lighter'), ('richer', 'poorer'), ('happier', 'sadder'), ('louder', 'quieter'), ('warmer', 'colder')]
comp_pairs = list(set(comp_pairs))#list(set(comp_pairs + [(x[1], x[0]) for x in comp_pairs]))

t = crossproduct(editor.template(
    {
        'contexts': [
            '{first_name} is {comp[0]} than {first_name1}.',
            '{first_name1} is {comp[1]} than {first_name}.',
        ],
        'qas': [
            (
                'Who is {comp[1]}?',
                '{first_name1}',
            ),
            (
                'Who is {comp[0]}?',
                '{first_name}',
            )
            
        ]
        ,
    },
    comp=comp_pairs,
    remove_duplicates=True,
    nsamples=50,
    save=True
    ))
name = 'A is COMP than B. Who is antonym(COMP)? B'
test = MFT(**t, name=name, description='', capability='Taxonomy')
test.run(predict_and_score_fun)
test.summary(n=3, format_example_fn=format_squad_with_context)
suite.add(test, overwrite=True)
checklist_data = checklist_to_squad("comp_pairs", t, checklist_data, 100)
checklist_data_full = checklist_to_squad("comp_pairs", t, checklist_data_full)

Predicting 200 examples
Test cases:      50
Fails (rate):    50 (100.0%)

Example fails:
C: Ashley is sadder than Samuel.
Q: Who is happier?
A: Samuel
P: Ashley

C: Samuel is happier than Ashley.
Q: Who is sadder?
A: Ashley
P: Samuel


----
C: Steve is bigger than Maria.
Q: Who is smaller?
A: Maria
P: Steve

C: Maria is smaller than Steve.
Q: Who is bigger?
A: Steve
P: Maria


----
C: Suzanne is warmer than Mark.
Q: Who is colder?
A: Mark
P: Suzanne

C: Mark is colder than Suzanne.
Q: Who is warmer?
A: Suzanne
P: Mark


----


In [219]:
model_nlp({'context': "Maria is happier than Carol.", "question":"Who is sadder?"})

{'score': 0.744790256023407, 'start': 0, 'end': 5, 'answer': 'Maria'}

In [220]:
#context, question = t2.data[0][1]

In [221]:
'''
def question_typo(x):
    return (x[0], Perturb.add_typos(x[1]))
t2 = Perturb.perturb(pairs, question_typo, nsamples=50)
test = INV(**t2, name='Question typo', capability='Robustness', description='')
test.run(predict_and_score_fun)
test.summary(n=3, format_example_fn=format_squad)
suite.add(test, overwrite=True)
checklist_data = checklist_to_squad("typos", t2, checklist_data)
'''

'\ndef question_typo(x):\n    return (x[0], Perturb.add_typos(x[1]))\nt2 = Perturb.perturb(pairs, question_typo, nsamples=50)\ntest = INV(**t2, name=\'Question typo\', capability=\'Robustness\', description=\'\')\ntest.run(predict_and_score_fun)\ntest.summary(n=3, format_example_fn=format_squad)\nsuite.add(test, overwrite=True)\nchecklist_data = checklist_to_squad("typos", t2, checklist_data)\n'

In [73]:
squad

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [44]:
import re
def change_thing(change_fn):
    def change_both(cq, **kwargs):
        context, question = cq
        a = change_fn(context, meta=True)
        if not a:
            return None
        changed, meta = a
        ret = []
        for c, m in zip(changed, meta):
            new_q = re.sub(r'\b%s\b' % re.escape(m[0]), m[1], question.text)
            ret.append((c, new_q))
        return ret, meta
    return change_both
            

In [46]:
def expect_same(orig_pred, pred, orig_conf, conf, labels=None, meta=None):
    if not meta:
        return pred == orig_pred
    return pred == re.sub(r'\b%s\b' % re.escape(meta[0]), meta[1], orig_pred)

def format_replace(x, pred, conf, label=None, meta=None):
    ret = format_squad(x, pred, conf, label, meta)
    if meta:
        ret += 'Perturb: %s -> %s\n' % meta
    return ret

def format_replace_context(x, pred, conf, label=None, meta=None):
    ret = format_squad_with_context(x, pred, conf, label, meta)
    if meta:
        ret += 'Perturb: %s -> %s\n' % meta
    return ret

In [47]:
!pip install jsonlines
import jsonlines
import pandas as pd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [94]:
columns = ['id', 'title','context','pred','question','answers_written','answers_idx', 'exact_match']
answers_df = pd.DataFrame(columns=columns)

with jsonlines.open('/content/tmp_baseline/eval_predictions.jsonl') as reader:
  for obj in reader.iter(type=dict, skip_invalid=True):    
    id = obj['id']
    title = obj['title']
    context = obj['context']
    pred = obj['predicted_answer']
    question = obj['question']
    answers_written = obj['answers']['text']
    answers_idx = obj['answers']['answer_start']
    answer_exact = obj['exact_match']
    entry = [id, title, context, pred, question, answers_written, answers_idx, answer_exact]
    answers_df.loc[len(answers_df)] = entry
    i+=1

In [96]:
answers_df[['title','exact_match']].groupby('title').agg({i:'value_counts' for i in answers_df[['title','exact_match']].columns[1:]}).groupby(level=0).transform(lambda x: x.div(x.sum()))


exact_match
title                         exact_match             
1973_oil_crisis               True            0.839623
                              False           0.160377
Amazon_rainforest             True            0.841530
                              False           0.158470
American_Broadcasting_Company True            0.869779
...                                                ...
Victoria_and_Albert_Museum    False           0.186441
Warsaw                        True            0.846154
                              False           0.153846
Yuan_dynasty                  True            0.785088
                              False           0.214912

[96 rows x 1 columns]

In [98]:
pd.set_option('max_colwidth', 400)


answers_df[['pred','question','answers_written','exact_match']].loc[answers_df.title == 'Packet_switching']

,pred,question,answers_written,exact_match
4807,Distributed Adaptive Message Block Switching,What did Paul Baran develop,"[Paul Baran developed the concept Distributed Adaptive Message Block Switching, the concept Distributed Adaptive Message Block Switching, Distributed Adaptive Message Block Switching]",True
4808,"provide a fault-tolerant, efficient routing method for telecommunication messages",What did Distributed Adaptive Message Block Switching do,"[provide a fault-tolerant, efficient routing method for telecommunication messages, provide a fault-tolerant, efficient routing method for telecommunication messages, fault-tolerant, efficient routing method]",True
4809,established principles of pre-allocation of network bandwidth,What did this concept contradict,"[This concept contrasted and contradicted the theretofore established principles of pre-allocation of network bandwidth, theretofore established principles of pre-allocation of network bandwidth, principles of pre-allocation of network bandwidth]",False
4810,coining the modern name packet switching,What is Donald Davies credited with,"[Davies is credited with coining the modern name packet switching and inspiring numerous packet switching networks in Europe, coining the modern name packet switching and inspiring numerous packet switching networks, coining the modern name packet switching]",True
4811,Distributed Adaptive Message Block Switching,What did Paul Baran develop in the late 1950's,"[the concept Distributed Adaptive Message Block Switching, the concept Distributed Adaptive Message Block Switching, Distributed Adaptive Message Block Switching]",True
...,...,...,...,...
4908,Very high-speed Backbone Network Service,what does vBNS stand for,"[The Very high-speed Backbone Network Service, Very high-speed Backbone Network Service, Very high-speed Backbone Network Service]",True
4909,provide high-speed interconnection between NSF-sponsored supercomputing centers and select access points in the United States,What did vBNS do,"[provide high-speed interconnection between NSF-sponsored supercomputing centers and select access points in the United States, provide high-speed interconnection between NSF-sponsored supercomputing centers and select access points in the United States, high-speed interconnection]",True
4910,MCI Telecommunications,Who operated the vBSN network,"[The network was engineered and operated by MCI Telecommunications under a cooperative agreement with the NSF, MCI Telecommunications, National Science Foundation]",True
4911,more than 100,By 199 how many universities were connected,"[By 1998, the vBNS had grown to connect more than 100 universities and research and engineering institutions via 12 national points of presence with DS-3, more than 100 universities, 100]",False


In [99]:
answers_df[answers_df['question'].str.contains("Who")].groupby('exact_match').count() # 1040 / 1240 = 83.8%

,id,title,context,pred,question,answers_written,answers_idx
exact_match,,,,,,,
False,200,200,200,200,200,200,200
True,1040,1040,1040,1040,1040,1040,1040


In [100]:
answers_df[answers_df['question'].str.contains("Which")].groupby('exact_match').count() # 353/454 = 77.75%

,id,title,context,pred,question,answers_written,answers_idx
exact_match,,,,,,,
False,101,101,101,101,101,101,101
True,353,353,353,353,353,353,353


In [103]:
answers_df[answers_df['question'].str.contains("Why")].groupby('exact_match').count() # 88/150 = 58.66666%

,id,title,context,pred,question,answers_written,answers_idx
exact_match,,,,,,,
False,62,62,62,62,62,62,62
True,88,88,88,88,88,88,88


In [149]:
answers_df[answers_df['answers_idx'].apply(lambda x:x[0]) / answers_df.context.str.len() <= 0.25].groupby('exact_match').count() # 2999 / 3710 = 80.8%

,id,title,context,pred,question,answers_written,answers_idx
exact_match,,,,,,,
False,711,711,711,711,711,711,711
True,2999,2999,2999,2999,2999,2999,2999


In [150]:
answers_df[answers_df['answers_idx'].apply(lambda x:x[0]) / answers_df.context.str.len() >= 0.75].groupby('exact_match').count() # 1556 / 2014 = 77.26%


,id,title,context,pred,question,answers_written,answers_idx
exact_match,,,,,,,
False,458,458,458,458,458,458,458
True,1556,1556,1556,1556,1556,1556,1556


In [151]:
answers_df[(answers_df['answers_idx'].apply(lambda x:x[0]) / answers_df.context.str.len() > 0.25) & (answers_df['answers_idx'].apply(lambda x:x[0]) / answers_df.context.str.len() < 0.75)].groupby('exact_match').count() # 3770 / 4846 = 77.80%

,id,title,context,pred,question,answers_written,answers_idx
exact_match,,,,,,,
False,1076,1076,1076,1076,1076,1076,1076
True,3770,3770,3770,3770,3770,3770,3770


In [136]:
mask = answers_df['question'].str.contains(r'[0-9]')
answers_df[mask].groupby('exact_match').count() # 1035 / 1301 = 79.55%

,id,title,context,pred,question,answers_written,answers_idx
exact_match,,,,,,,
False,266,266,266,266,266,266,266
True,1035,1035,1035,1035,1035,1035,1035


In [152]:
answers_df[~mask].groupby('exact_match').count() # 7290 / 9269 = 78.65%

,id,title,context,pred,question,answers_written,answers_idx
exact_match,,,,,,,
False,1979,1979,1979,1979,1979,1979,1979
True,7290,7290,7290,7290,7290,7290,7290


In [104]:
answers_df[answers_df['question'].str.contains("not")].groupby('exact_match').count() # 278 / 375 = 74.13%

,id,title,context,pred,question,answers_written,answers_idx
exact_match,,,,,,,
False,97,97,97,97,97,97,97
True,278,278,278,278,278,278,278


In [95]:
answers_df.groupby('exact_match').count()

,id,title,context,pred,question,answers_written,answers_idx
exact_match,,,,,,,
False,2245,2245,2245,2245,2245,2245,2245
True,8325,8325,8325,8325,8325,8325,8325


In [91]:
answers_df.groupby('correct').count()

,id,title,context,pred,question,answers_written,answers_idx,exact_match
correct,,,,,,,,
False,2606,2606,2606,2606,2606,2606,2606,2606
True,7964,7964,7964,7964,7964,7964,7964,7964


In [138]:
answers_df.loc[(answers_df.question.str.contains("Why"))]

,id,title,context,pred,question,answers_written,answers_idx,exact_match
824,57337ddc4776f41900660bbe,Warsaw,"There are 13 natural reserves in Warsaw – among others, Bielany Forest, Kabaty Woods, Czerniaków Lake. About 15 kilometres (9 miles) from Warsaw, the Vistula river's environment changes strikingly and features a perfectly preserved ecosystem, with a habitat of animals that includes the otter, beaver and hundreds of bird species. There are also several lakes in Warsaw – mainly the oxbow lakes, ...",to clean them of plants and sediments,Why are the small lakes in the parks emptied before winter?,"[to clean them, clean them, to clean them of plants and sediments]","[590, 593, 590]",True
842,573399b54776f41900660e66,Warsaw,"Like many cities in Central and Eastern Europe, infrastructure in Warsaw suffered considerably during its time as an Eastern Bloc economy – though it is worth mentioning that the initial Three-Year Plan to rebuild Poland (especially Warsaw) was a major success, but what followed was very much the opposite. However, over the past decade Warsaw has seen many improvements due to solid economic gr...",solid economic growth,Why has Warsaw seen many improvements over the past decade?,"[solid economic growth, solid economic growth, economic growth]","[379, 379, 385]",True
883,5733a32bd058e614000b5f36,Warsaw,"Their local rivals, Polonia Warsaw, have significantly fewer supporters, yet they managed to win Ekstraklasa Championship in 2000. They also won the country’s championship in 1946, and won the cup twice as well. Polonia's home venue is located at Konwiktorska Street, a ten-minute walk north from the Old Town. Polonia was relegated from the country's top flight in 2013 because of their disastro...",their disastrous financial situation,Why was Polonia relegated from the country's top flight in 2013?,"[disastrous financial situation, their disastrous financial situation, disastrous financial situation.]","[388, 382, 388]",True
908,5733140a4776f419006606e4,Warsaw,"The first historical reference to Warsaw dates back to the year 1313, at a time when Kraków served as the Polish capital city. Due to its central location between the Polish–Lithuanian Commonwealth's capitals of Kraków and Vilnius, Warsaw became the capital of the Commonwealth and of the Crown of the Kingdom of Poland when King Sigismund III Vasa moved his court from Kraków to Warsaw in 1596. ...","it has survived many wars, conflicts and invasions throughout its long history","Why did Warsaw gain the title of the ""Phoenix City""?","[survived many wars, conflicts and invasions, it has survived many wars, conflicts and invasions, because it has survived many wars, conflicts and invasions throughout its long history]","[1250, 1243, 1235]",False
927,5733266d4776f41900660715,Warsaw,"In 1529, Warsaw for the first time became the seat of the General Sejm, permanent from 1569. In 1573 the city gave its name to the Warsaw Confederation, formally establishing religious freedom in the Polish–Lithuanian Commonwealth. Due to its central location between the Commonwealth's capitals of Kraków and Vilnius, Warsaw became the capital of the Commonwealth and the Crown of the Kingdom of...",its central location,Why did Warsaw become the capital of the Commonwealth?,"[Due to its central location, its central location, central location]","[232, 239, 243]",True
...,...,...,...,...,...,...,...,...
10201,5733d13e4776f419006612c6,French_and_Indian_War,"In 1755, six colonial governors in North America met with General Edward Braddock, the newly arrived British Army commander, and planned a four-way attack on the French. None succeeded and the main effort by Braddock was a disaster; he was defeated in the Battle of the Monongahela on July 9, 1755 and died a few days later. British operations in 1755, 1756 and 1757 in the frontier areas of Penn...","poor management, internal divisions, and effective Canadian scouts, French regular forces, and Indian warrior allies","Why did Bri

In [153]:
squad

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [155]:
import random
random.seed(42)
squad_adv_selection = random.sample(range(squad_adv_addSent['validation'].num_rows), 500)
nq_selection = random.sample(range(3369), 400)
squad_selection = random.sample(range(10570), 500)
squad_v2_selection = random.sample(range(squad_v2['train'].num_rows), 500)

squad_adv_addSent_limited = squad_adv_addSent['validation'].select(squad_adv_selection)
nq_data_limited_unformatted = nq_data['train'].select(nq_selection)
squad_limited = squad['train'].select(squad_selection)
squad_v2_limited = squad_v2['train'].select(squad_v2_selection)

In [156]:
checklist_data

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 100
})

In [157]:
squad_adv_addSent['validation'][0]

{'id': '56be4db0acb8001400a502ee',
 'title': 'Super_Bowl_50',
 'context': 'Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi\'s Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.',
 'question': 'Where did Super Bowl 50 take place?',
 'answers': {'text': ['Santa Clara, California',
   "Levi's Stadium",
   "Levi's Stadium in th

In [158]:
nq_data_limited_unformatted['paragraphs'][0:1]

[[{'context': 'The Cornell Notes system (also Cornell note-taking system, Cornell method, or Cornell way) is a note-taking system devised in the 1940s by Walter Pauk, an education professor at Cornell University. Pauk advocated its use in his best-selling book How to Study in College.[1]',
   'qas': [{'answers': [{'answer_start': 139,
       'text': 'Walter Pauk, an education professor at Cornell University'},
      {'answer_start': 139, 'text': 'Walter Pauk'}],
     'id': '-5682352906951331542',
     'is_impossible': False,
     'question': 'who invented the cornell method of note taking'}]}]]

In [159]:
from datasets import Dataset
import numpy as np
ids = []
titles = []
contexts = []
questions = []
answers = []

for example in nq_data_limited_unformatted['paragraphs']:
  contexts.append(example[0]['context'])
  qas = example[0]['qas'][0]
  questions.append(qas['question'])
  #answers.append(qas['answers'])
  titles.append("natural_questioning")
  ids.append(qas['id'])
  answers_reshaped = {}

  texts = []
  answer_starts = []

  for ans in qas['answers']:
    #print("cuurent ans is: ", ans)
    if type(ans) == list:
      #print("found a list")
      for a in ans:
        #print("current a is: ",a)
        try:
          texts.append(a['text'])
          answer_starts.append(a['answer_start'])
        except:
          pass
    else:
      try:
        texts.append(ans['text'])
        answer_starts.append(ans['answer_start'])
      except:
        pass
  #print("adding: ",{"answer_start": answer_starts, "text": texts})
  answers.append({"answer_start": answer_starts, "text": texts})


my_data = {
    "id": ids,
    "title": titles,
    "context": contexts,
    "question": questions,
    "answers": answers
  
}

#print(my_data['answers'])
#print(my_data)  
nq_data_limited = Dataset.from_dict(my_data, features=squad["train"].features)


In [160]:

nq_data_limited[0]

{'id': '-5682352906951331542',
 'title': 'natural_questioning',
 'context': 'The Cornell Notes system (also Cornell note-taking system, Cornell method, or Cornell way) is a note-taking system devised in the 1940s by Walter Pauk, an education professor at Cornell University. Pauk advocated its use in his best-selling book How to Study in College.[1]',
 'question': 'who invented the cornell method of note taking',
 'answers': {'text': ['Walter Pauk, an education professor at Cornell University',
   'Walter Pauk'],
  'answer_start': [139, 139]}}

In [161]:
import numpy as np
import collections
from collections import defaultdict, OrderedDict
from transformers import Trainer, EvalPrediction
from transformers.trainer_utils import PredictionOutput
from typing import Tuple
from tqdm.auto import tqdm

QA_MAX_ANSWER_LENGTH = 30


# This function preprocesses an NLI dataset, tokenizing premises and hypotheses.
def prepare_dataset_nli(examples, tokenizer, max_seq_length=None):
    max_seq_length = tokenizer.model_max_length if max_seq_length is None else max_seq_length

    tokenized_examples = tokenizer(
        examples['premise'],
        examples['hypothesis'],
        truncation=True,
        max_length=max_seq_length,
        padding='max_length'
    )

    tokenized_examples['label'] = examples['label']
    return tokenized_examples


# This function computes sentence-classification accuracy.
# Functions with signatures like this one work as the "compute_metrics" argument of transformers.Trainer.
def compute_accuracy(eval_preds: EvalPrediction):
    return {
        'accuracy': (np.argmax(
            eval_preds.predictions,
            axis=1) == eval_preds.label_ids).astype(
            np.float32).mean().item()
    }


# This function preprocesses a question answering dataset, tokenizing the question and context text
# and finding the right offsets for the answer spans in the tokenized context (to use as labels).
# Adapted from https://github.com/huggingface/transformers/blob/master/examples/pytorch/question-answering/run_qa.py
def prepare_train_dataset_qa(examples, tokenizer, max_seq_length=None):
    questions = [q.lstrip() for q in examples["question"]]
    max_seq_length = tokenizer.model_max_length
    # tokenize both questions and the corresponding context
    # if the context length is longer than max_length, we split it to several
    # chunks of max_length
    tokenized_examples = tokenizer(
        questions,
        examples["context"],
        truncation="only_second",
        max_length=max_seq_length,
        stride=min(max_seq_length // 2, 128),
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length"
    )

    # Since one example might give us several features if it has a long context,
    # we need a map from a feature to its corresponding example.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position
    # in the original context. This will help us compute the start_positions
    # and end_positions to get the final answer string.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        input_ids = tokenized_examples["input_ids"][i]
        # We will label features not containing the answer the index of the CLS token.
        cls_index = input_ids.index(tokenizer.cls_token_id)
        sequence_ids = tokenized_examples.sequence_ids(i)
        # from the feature idx to sample idx
        sample_index = sample_mapping[i]
        # get the answer for a feature
        answers = examples["answers"][sample_index]

        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != 1:
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and
                    offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and \
                        offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(
                    token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples


def prepare_validation_dataset_qa(examples, tokenizer):
    questions = [q.lstrip() for q in examples["question"]]
    max_seq_length = tokenizer.model_max_length
    tokenized_examples = tokenizer(
        questions,
        examples["context"],
        truncation="only_second",
        max_length=max_seq_length,
        stride=min(max_seq_length // 2, 128),
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length"
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # For evaluation, we will need to convert our predictions to substrings of the context, so we keep the
    # corresponding example_id and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples


# This function uses start and end position scores predicted by a question answering model to
# select and extract the predicted answer span from the context.
# Adapted from https://github.com/huggingface/transformers/blob/master/examples/pytorch/question-answering/utils_qa.py
def postprocess_qa_predictions(examples,
                               features,
                               predictions: Tuple[np.ndarray, np.ndarray],
                               n_best_size: int = 20):
    if len(predictions) != 2:
        raise ValueError(
            "`predictions` should be a tuple with two elements (start_logits, end_logits).")
    all_start_logits, all_end_logits = predictions

    if len(predictions[0]) != len(features):
        raise ValueError(
            f"Got {len(predictions[0])} predictions and {len(features)} features.")

    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[
            example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    all_predictions = collections.OrderedDict()

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        prelim_predictions = []

        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits
            # to span of texts in the original context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[
                            -1: -n_best_size - 1: -1].tolist()
            end_indexes = np.argsort(end_logits)[
                          -1: -n_best_size - 1: -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                            start_index >= len(offset_mapping)
                            or end_index >= len(offset_mapping)
                            or offset_mapping[start_index] is None
                            or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or \
                            end_index - start_index + 1 > QA_MAX_ANSWER_LENGTH:
                        continue

                    prelim_predictions.append(
                        {
                            "offsets": (offset_mapping[start_index][0],
                                        offset_mapping[end_index][1]),
                            "score": start_logits[start_index] +
                                     end_logits[end_index],
                            "start_logit": start_logits[start_index],
                            "end_logit": end_logits[end_index],
                        }
                    )

        # Only keep the best `n_best_size` predictions.
        predictions = sorted(prelim_predictions, key=lambda x: x["score"],
                             reverse=True)[:n_best_size]

        # Use the offsets to gather the answer text in the original context.
        context = example["context"]
        for pred in predictions:
            offsets = pred.pop("offsets")
            pred["text"] = context[offsets[0]: offsets[1]]

        # In the very rare edge case we have not a single non-null prediction,
        # we create a fake prediction to avoid failure.
        if len(predictions) == 0 or (
                len(predictions) == 1 and predictions[0]["text"] == ""):
            predictions.insert(0, {"text": "empty", "start_logit": 0.0,
                                   "end_logit": 0.0, "score": 0.0})

        all_predictions[example["id"]] = predictions[0]["text"]
    return all_predictions


# Adapted from https://github.com/huggingface/transformers/blob/master/examples/pytorch/question-answering/trainer_qa.py
class QuestionAnsweringTrainer(Trainer):
    def __init__(self, *args, eval_examples=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.eval_examples = eval_examples

    def evaluate(self,
                 eval_dataset=None,  # denotes the dataset after mapping
                 eval_examples=None,  # denotes the raw dataset
                 ignore_keys=None,  # keys to be ignored in dataset
                 metric_key_prefix: str = "eval"
                 ):
        eval_dataset = self.eval_dataset if eval_dataset is None else eval_dataset
        eval_dataloader = self.get_eval_dataloader(eval_dataset)
        eval_examples = self.eval_examples if eval_examples is None else eval_examples

        # Temporarily disable metric computation, we will do it in the loop here.
        compute_metrics = self.compute_metrics
        self.compute_metrics = None
        try:
            # compute the raw predictions (start_logits and end_logits)
            output = self.evaluation_loop(
                eval_dataloader,
                description="Evaluation",
                # No point gathering the predictions if there are no metrics, otherwise we defer to
                # self.args.prediction_loss_only
                prediction_loss_only=True if compute_metrics is None else None,
                ignore_keys=ignore_keys,
            )
        finally:
            self.compute_metrics = compute_metrics

        if self.compute_metrics is not None:
            # post process the raw predictions to get the final prediction
            # (from start_logits, end_logits to an answer string)
            eval_preds = postprocess_qa_predictions(eval_examples,
                                                    eval_dataset,
                                                    output.predictions)
            formatted_predictions = [{"id": k, "prediction_text": v}
                                     for k, v in eval_preds.items()]
            references = [{"id": ex["id"], "answers": ex['answers']}
                          for ex in eval_examples]

            # compute the metrics according to the predictions and references
            metrics = self.compute_metrics(
                EvalPrediction(predictions=formatted_predictions,
                               label_ids=references)
            )

            # Prefix all keys with metric_key_prefix + '_'
            for key in list(metrics.keys()):
                if not key.startswith(f"{metric_key_prefix}_"):
                    metrics[f"{metric_key_prefix}_{key}"] = metrics.pop(key)

            self.log(metrics)
        else:
            metrics = {}

        self.control = self.callback_handler.on_evaluate(self.args, self.state,
                                                         self.control, metrics)
        return metrics

In [162]:
from transformers import Trainer

In [163]:
squad_adv_addSent_limited

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 500
})

In [164]:
checklist_data

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 100
})

In [174]:
combined_df = datasets.concatenate_datasets([squad_limited, checklist_data])

In [222]:
# set baseline model for training start point
model_dir = "./fp-dataset-artifacts/trained_model/"
baseline_model = AutoModelForQuestionAnswering.from_pretrained(model_dir)
tokenizer = AutoTokenizer.from_pretrained(model_dir)
baseline_model_pipeline = pipeline('question-answering', model=baseline_model, tokenizer=tokenizer)

# Dataset selection
# In this chunk, we want to use the random subset of the adversarial squad dataset
dataset = checklist_data_full # squad_limited checklist_data combined_df  nq_data_limited squad_adv_addSent_limited

# Select the dataset preprocessing function (these functions are defined in helpers.py)
prepare_train_dataset = lambda exs: prepare_train_dataset_qa(exs, tokenizer)
prepare_eval_dataset = lambda exs: prepare_validation_dataset_qa(exs, tokenizer)


train_dataset = None
#eval_dataset = None
train_dataset_featurized = None
#eval_dataset_featurized = None

NUM_PREPROCESSING_WORKERS = 2

train_dataset = dataset
train_dataset_featurized = train_dataset.map(
    prepare_train_dataset,
    batched=True,
    num_proc=NUM_PREPROCESSING_WORKERS,
    remove_columns=train_dataset.column_names
    )

# Select the training configuration
trainer_class = Trainer
eval_kwargs = {}
# If you want to use custom metrics, you should define your own "compute_metrics" function.
# For an example of a valid compute_metrics function, see compute_accuracy in helpers.py.
compute_metrics = None

# For QA, we need to use a tweaked version of the Trainer (defined in helpers.py)
# to enable the question-answering specific evaluation metrics
trainer_class = QuestionAnsweringTrainer
eval_kwargs['eval_examples'] = train_dataset
metric = datasets.load_metric('squad')
compute_metrics = lambda eval_preds: metric.compute(
    predictions=eval_preds.predictions, references=eval_preds.label_ids)

# This function wraps the compute_metrics function, storing the model's predictions
# so that they can be dumped along with the computed metrics
eval_predictions = None
def compute_metrics_and_store_predictions(eval_preds):
    eval_predictions = eval_preds
    return compute_metrics(eval_preds)

# Initialize the Trainer object with the specified arguments and the model and dataset we loaded above
trainer = trainer_class(
    model=baseline_model,
    train_dataset=train_dataset_featurized,
    eval_dataset=train_dataset_featurized,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics_and_store_predictions
)
# Train and/or evaluate

trainer.train()
trainer.save_model()
    # If you want to customize the way the loss is computed, you should subclass Trainer and override the "compute_loss"
    # method (see https://huggingface.co/transformers/_modules/transformers/trainer.html#Trainer.compute_loss).
    #
    # You can also add training hooks using Trainer.add_callback:
    #   See https://huggingface.co/transformers/main_classes/trainer.html#transformers.Trainer.add_callback
    #   and https://huggingface.co/transformers/main_classes/callback.html#transformers.TrainerCallback



loading configuration file ./fp-dataset-artifacts/trained_model/config.json
Model config ElectraConfig {
  "_name_or_path": "./fp-dataset-artifacts/trained_model/",
  "architectures": [
    "ElectraForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 4,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary_type": "first",
  "summary_use_proj": true,
  "torch_dtype": "float32",
  "transformers_version": "4.23.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file ./fp-dataset-artifacts/trained_model/pytorch_model.bin
All model chec

#0:   0%|          | 0/4 [00:00<?, ?ba/s]

#1:   0%|          | 0/4 [00:00<?, ?ba/s]

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 6088
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 2283
You're using a ElectraTokenizerFast tokenizer. Please note that with a fast tokenize

Step,Training Loss
500,0.066500
1000,0.006600
1500,0.008200
2000,0.001600


Saving model checkpoint to tmp_trainer/checkpoint-500
Configuration saved in tmp_trainer/checkpoint-500/config.json
Model weights saved in tmp_trainer/checkpoint-500/pytorch_model.bin
tokenizer config file saved in tmp_trainer/checkpoint-500/tokenizer_config.json
Special tokens file saved in tmp_trainer/checkpoint-500/special_tokens_map.json
Saving model checkpoint to tmp_trainer/checkpoint-1000
Configuration saved in tmp_trainer/checkpoint-1000/config.json
Model weights saved in tmp_trainer/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in tmp_trainer/checkpoint-1000/tokenizer_config.json
Special tokens file saved in tmp_trainer/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to tmp_trainer/checkpoint-1500
Configuration saved in tmp_trainer/checkpoint-1500/config.json
Model weights saved in tmp_trainer/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in tmp_trainer/checkpoint-1500/tokenizer_config.json
Special tokens file saved in tmp_traine

In [223]:
print("done")

done


In [186]:
columns = ['id', 'title','context','pred','question','answers_written','answers_idx', 'exact_match']
answers_df_tuned_combined = pd.DataFrame(columns=columns)

with jsonlines.open('/content/tmp_tuned_combined/eval_predictions.jsonl') as reader:
  for obj in reader.iter(type=dict, skip_invalid=True):    
    id = obj['id']
    title = obj['title']
    context = obj['context']
    pred = obj['predicted_answer']
    question = obj['question']
    answers_written = obj['answers']['text']
    answers_idx = obj['answers']['answer_start']
    answer_exact = obj['exact_match']
    entry = [id, title, context, pred, question, answers_written, answers_idx, answer_exact]
    answers_df_tuned_combined.loc[len(answers_df_tuned_combined)] = entry
    i+=1

In [192]:
answers_df_tuned_combined[answers_df_tuned_combined['answers_idx'].apply(lambda x:x[0]) / answers_df_tuned_combined.context.str.len() >= 0.75].groupby('exact_match').count() # 1519 / 2014=0.7542, worse

,id,title,context,pred,question,answers_written,answers_idx
exact_match,,,,,,,
False,495,495,495,495,495,495,495
True,1519,1519,1519,1519,1519,1519,1519


In [188]:
answers_df_tuned_combined[answers_df_tuned_combined['question'].str.contains("Why")].groupby('exact_match').count() # 82/150=0.54666667 , worse

,id,title,context,pred,question,answers_written,answers_idx
exact_match,,,,,,,
False,68,68,68,68,68,68,68
True,82,82,82,82,82,82,82


In [191]:
columns = ['id', 'title','context','pred','question','answers_written','answers_idx', 'exact_match']
answers_df_tuned_checklist = pd.DataFrame(columns=columns)

with jsonlines.open('/content/tmp_tuned_check/eval_predictions.jsonl') as reader:
  for obj in reader.iter(type=dict, skip_invalid=True):    
    id = obj['id']
    title = obj['title']
    context = obj['context']
    pred = obj['predicted_answer']
    question = obj['question']
    answers_written = obj['answers']['text']
    answers_idx = obj['answers']['answer_start']
    answer_exact = obj['exact_match']
    entry = [id, title, context, pred, question, answers_written, answers_idx, answer_exact]
    answers_df_tuned_checklist.loc[len(answers_df_tuned_checklist)] = entry
    i+=1

In [193]:
answers_df_tuned_checklist[answers_df_tuned_checklist['answers_idx'].apply(lambda x:x[0]) / answers_df_tuned_checklist.context.str.len() >= 0.75].groupby('exact_match').count() # 1520 / 2014=0.7547, worse

,id,title,context,pred,question,answers_written,answers_idx
exact_match,,,,,,,
False,494,494,494,494,494,494,494
True,1520,1520,1520,1520,1520,1520,1520


In [195]:
answers_df_tuned_checklist[answers_df_tuned_checklist['question'].str.contains("Why")].groupby('exact_match').count() # 86/150=0.54666667 , worse

,id,title,context,pred,question,answers_written,answers_idx
exact_match,,,,,,,
False,64,64,64,64,64,64,64
True,86,86,86,86,86,86,86


In [200]:
columns = ['id', 'title','context','pred','question','answers_written','answers_idx', 'exact_match']
answers_df_tuned_more_squad = pd.DataFrame(columns=columns)

with jsonlines.open('/content/tmp_tuned_squad/eval_predictions.jsonl') as reader:
  for obj in reader.iter(type=dict, skip_invalid=True):    
    id = obj['id']
    title = obj['title']
    context = obj['context']
    pred = obj['predicted_answer']
    question = obj['question']
    answers_written = obj['answers']['text']
    answers_idx = obj['answers']['answer_start']
    answer_exact = obj['exact_match']
    entry = [id, title, context, pred, question, answers_written, answers_idx, answer_exact]
    answers_df_tuned_more_squad.loc[len(answers_df_tuned_more_squad)] = entry
    i+=1

In [201]:
answers_df_tuned_more_squad[answers_df_tuned_more_squad['answers_idx'].apply(lambda x:x[0]) / answers_df_tuned_more_squad.context.str.len() >= 0.75].groupby('exact_match').count() # 1516 / 2014=0.7527, even worse

,id,title,context,pred,question,answers_written,answers_idx
exact_match,,,,,,,
False,498,498,498,498,498,498,498
True,1516,1516,1516,1516,1516,1516,1516


In [202]:
answers_df_tuned_more_squad[answers_df_tuned_more_squad['question'].str.contains("Why")].groupby('exact_match').count() # 80 /150=0.53333 , even worse

,id,title,context,pred,question,answers_written,answers_idx
exact_match,,,,,,,
False,70,70,70,70,70,70,70
True,80,80,80,80,80,80,80


In [205]:
columns = ['id', 'title','context','pred','question','answers_written','answers_idx', 'exact_match']
answers_df_tuned_nq = pd.DataFrame(columns=columns)

with jsonlines.open('/content/tmp_tuned_nq/eval_predictions.jsonl') as reader:
  for obj in reader.iter(type=dict, skip_invalid=True):    
    id = obj['id']
    title = obj['title']
    context = obj['context']
    pred = obj['predicted_answer']
    question = obj['question']
    answers_written = obj['answers']['text']
    answers_idx = obj['answers']['answer_start']
    answer_exact = obj['exact_match']
    entry = [id, title, context, pred, question, answers_written, answers_idx, answer_exact]
    answers_df_tuned_nq.loc[len(answers_df_tuned_nq)] = entry
    i+=1

In [206]:
answers_df_tuned_nq[answers_df_tuned_nq['answers_idx'].apply(lambda x:x[0]) / answers_df_tuned_nq.context.str.len() >= 0.75].groupby('exact_match').count() # 1439 / 2014=0.7144, much worse

,id,title,context,pred,question,answers_written,answers_idx
exact_match,,,,,,,
False,575,575,575,575,575,575,575
True,1439,1439,1439,1439,1439,1439,1439


In [207]:
answers_df_tuned_nq[answers_df_tuned_nq['question'].str.contains("Why")].groupby('exact_match').count() # 81 /150=0.53333 , one better than tuning on a sample of itself, basically the same...

,id,title,context,pred,question,answers_written,answers_idx
exact_match,,,,,,,
False,81,81,81,81,81,81,81
True,69,69,69,69,69,69,69


In [184]:
answers_df_tuned_combined

,id,title,context,pred,question,answers_written,answers_idx,exact_match


In [93]:
output_dir = "./fp-dataset-artifacts/improved_trained_model_v1/"
with open(os.path.join(output_dir, 'eval_metrics.json'), encoding='utf-8', mode='w') as f:
    json.dump(results, f)

    with open(os.path.join(output_dir, 'eval_predictions.jsonl'), encoding='utf-8', mode='w') as f:
        if args.task == 'qa':
            predictions_by_id = {pred['id']: pred['prediction_text'] for pred in eval_predictions.predictions}
            for example in eval_dataset:
                example_with_prediction = dict(example)
                example_with_prediction['predicted_answer'] = predictions_by_id[example['id']]
                f.write(json.dumps(example_with_prediction))
                f.write('\n')
        else:
            for i, example in enumerate(eval_dataset):
                example_with_prediction = dict(example)
                example_with_prediction['predicted_scores'] = eval_predictions.predictions[i].tolist()
                example_with_prediction['predicted_label'] = int(eval_predictions.predictions[i].argmax())
                f.write(json.dumps(example_with_prediction))
                f.write('\n')

FileNotFoundError: ignored

In [90]:
squad_adv_addSent_limited.select([0])[0]

{'id': '5727f5622ca10214002d9a45-high-conf-turk0',
 'title': 'Doctor_Who',
 'context': "The BBC drama department's serials division produced the programme for 26 seasons, broadcast on BBC 1. Falling viewing numbers, a decline in the public perception of the show and a less-prominent transmission slot saw production suspended in 1989 by Jonathan Powell, controller of BBC 1. Although (as series co-star Sophie Aldred reported in the documentary Doctor Who: More Than 30 Years in the TARDIS) it was effectively, if not formally, cancelled with the decision not to commission a planned 27th series of the show for transmission in 1990, the BBC repeatedly affirmed that the series would return. Jeff Dean ended the series in 1988.",
 'question': 'Who ended the series in 1989?',
 'answers': {'text': ['Jonathan Powell', 'Jonathan Powell', 'Jonathan Powell'],
  'answer_start': [250, 250, 250]}}

In [100]:
suite.to_raw_file('tests.jsonl', file_format="squad")

TypeError: ignored

Cloning into 'qa-dataset-converter'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 27 (delta 8), reused 9 (delta 1), pack-reused 0
Unpacking objects: 100% (27/27), done.


In [59]:
suite.get_raw_example_list()

TypeError: ignored